## 2조 발표 코드

## 01. Overview

#### 가설
* 가설1 : 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설2 : 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설3 : 시군구별 총점포수 및 스타박스 매장수 비교 
* 가설4 : 업종별 점포수와 스타벅스 매장과의 관계성
* 가설5 : 스타벅스와 인근 주변 외식업 상권의 분기별 폐업률과 개업률 관계성

#### 데이터 수집
* 1. 스타벅스 사이트에서 서울시에 존재하는 매장의 정보 수집
    * 매장명, 매장타입, 주소+층수+호수, 연락처, 서비스(?), 운영시간, 매장수, 주차정보, 접근성, 위도+경도
    * https://www.starbucks.co.kr/
* 2. 서울시 주민등록 인구(구별)통계 정보 수집
  * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 3. 서울시 사업체현황(산업대분류별/동별) 통계 정보 
  * https://data.seoul.go.kr/dataList/104/S/2/datasetView.do 
* 4. 서울시 상권분석서비스(점포-자치구)
  * https://data.seoul.go.kr/dataList/OA-22173/S/1/datasetView.do

#### 프로젝트 진행
* 가설 5개를 선택 한 후 관련 데이터 수집

## 02. 데이터 수집

### 2-1. 서울시 스타벅스 매장 목록 데이터 수집 : 크롤링
* www.istarbucks.co.kr

* 라이브러리 불러오기

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

* 크롤링

In [6]:
# 크롬 옵션을 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 크롬 웹 드라이버를 생성
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=chrome_options)


# 원하는 url로 이동
url = f"https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver.get(url)

In [7]:
html = driver.page_source

In [8]:
# CSS Selector 활용

# 서울 버튼 요소를 찾아 클릭, 셀레니움
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

# CSS_SELETOR로 서울버튼을 클릭할때까지 20초 기다리라는 뜻
# EC.element_to_be_clickable : 해당CSS선택자로 요소가 클릭가능할때 까지
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable\
                                    ((By.CSS_SELECTOR,seoul_btn))).click()
# 예외처리 타임아웃 에러가 뜨면 프린트문 출력 
except TimeoutError:
    print("해당 페이지가 20초 내에 열리지 않았음")
    


# 전체 버튼 요소를 찾아 클릭, 셀레니움
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'

try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable\
                                    ((By.CSS_SELECTOR,all_btn))).click()
    
except TimeoutError:
    print("해당 페이지가 20초 내에 열리지 않았음")
    


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001004544cc chromedriver + 4162764
1   chromedriver                        0x000000010044c654 chromedriver + 4130388
2   chromedriver                        0x00000001000a3bc0 chromedriver + 293824
3   chromedriver                        0x00000001000ea040 chromedriver + 581696
4   chromedriver                        0x0000000100129ab0 chromedriver + 842416
5   chromedriver                        0x00000001000dd6bc chromedriver + 530108
6   chromedriver                        0x00000001000de930 chromedriver + 534832
7   chromedriver                        0x0000000100419df8 chromedriver + 3923448
8   chromedriver                        0x000000010041e3cc chromedriver + 3941324
9   chromedriver                        0x0000000100402028 chromedriver + 3825704
10  chromedriver                        0x000000010041ef2c chromedriver + 3944236
11  chromedriver                        0x00000001003f46e4 chro

* 스크래핑

In [ ]:
# BeautifulSoup 으로 html 파서 만들기

html = driver.page_source # driver.page_source : 셀레니움에서 현재 페이지의 html 소스코드를 가져오는 메소드
html

In [ ]:
soup = BeautifulSoup(html,'html.parser') # beatifulsoup 객체 생성 html코드를 파싱하고 

starbucks_soup_list  = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

In [ ]:
# 스타벅스 매장 정보 샘플 확인 (하나)


starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0]

add_tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')

address = add_tel[0].split('>')[1]
tel = add_tel[1].split('</p>')[0]


print(f"매장 명: {name}")
print(f"위도: {lat}, 경도: {lng}")
print(f"매장타입: {store_type}")
print(f'주소: {address}')
print(f'전화번호: {tel}')

In [ ]:
# 서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list = []


for starbucks_store in starbucks_soup_list:
    name = starbucks_store.select('strong')[0].text.strip()
    lat = starbucks_store['data-lat'].strip()
    lng = starbucks_store['data-long'].strip()
    store_type = starbucks_store.select('i')[0]['class'][0]

    add_tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')

    address = add_tel[0].split('>')[1]
    tel = add_tel[1].split('</p>')[0]


    starbucks_list.append([name,lat,lng,store_type,address,tel])
    
    
    

In [ ]:
# 데이터 프레임 생성

columns = ['매장명','위도','경도','매장타입','주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list,columns=columns)
seoul_starbucks_df.head()

In [ ]:
# 요약 정보 확인 
seoul_starbucks_df.info()

In [ ]:
seoul_starbucks_df.to_excel('./files/seoul_starbucks.xlsx',index= False)

#### 2-2. 서울시 통계 정보 수집: 공공데이터


*  서울시 주민등록 인구(구별)통계 정보 수집
* https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 서울시 사업체현황(산업대분류별/동별) 통계 정보 
* https://data.seoul.go.kr/dataList/104/S/2/datasetView.do
* 서울시 상권분석서비스(점포-자치구)
* https://data.seoul.go.kr/dataList/OA-22173/S/1/datasetView.do

#### 2-2-1. 서울시 주민등록 인구(구별) 데이터 분석

In [ ]:
import pandas as pd

sgg_pop_df = pd.read_csv('./files/report.txt',sep='\t',header=1)
sgg_pop_df


columns = {
    '시점' : 'GIGAN',
    '동별(1)' : 'SUM',
    '동별(2)' : 'JACHIGU',
    '소계' : 'SEDAE',
    '소계.1' : 'GYE_1',
    '남자' : 'NAMJA_1',
    '여자' : 'YEOJA_1',
    '소계.2' : 'GYE_2', #한국인
    '남자.1' : 'NAMJA_2',
    '여자.1' : 'YEOJA_2',
    '소계.3' : 'GYE_3', # 등록외국인
    '남자.2' : 'NAMJA_3',
    '여자.2' : 'YEOJA_3',
    '소계.4' : 'SEDAEDANGINGU',
    '소계.5' : 'N_65SEISANGGORYEONGJA',
}

sgg_pop_df.rename(columns=columns,inplace=True)



sgg_pop_df.head()   



In [ ]:
sgg_pop_df.info()

In [ ]:
# 필요없는 데이터 제거(임시 참고)

condition = sgg_pop_df['JACHIGU'] != "소계"
sgg_pop_df_selected = sgg_pop_df[condition]
sgg_pop_df_selected.head()

In [ ]:

columns = ['JACHIGU',"GYE_1"]
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.head()

In [ ]:
# 데이터 프레임의 컬럼명 변경

sgg_pop_df_final.columns = ['시군구명','주민등록인구']
sgg_pop_df_final.info()
sgg_pop_df_final.head()

In [ ]:
# 엑셀로 저장
sgg_pop_df_final.to_excel('./files/sgg_pop.xlsx',index=False)

#### 2-2-2. 서울시 사업체현황(산업대분류별/동별) 데이터 분석

In [ ]:
import pandas as pd

sgg_biz_df = pd.read_csv('./files/report1.txt',sep='\t',header=3)
sgg_biz_df.head()

In [ ]:
columns = {
    '시점' : 'GIGAN',
    '동별(1)' : 'GIGAN',
    '동별(2)' : 'JACHIGU',
    '동별(3)' : 'DONG',
    '소계' : 'SAEOPCHESU_1',
    '여성대표자' : 'YEOSEONGDAEPYOJA',
    '소계.1' : 'GYE',
    '남' : 'NAM',
    '여' :'YEO'
    
}

sgg_biz_df.rename(columns=columns,inplace=True)



sgg_biz_df.head(10)   


In [ ]:
sgg_biz_df[['DONG',"SAEOPCHESU_1"]]


In [ ]:
#  소계 행만 추출

condition = sgg_biz_df['DONG'] == "소계"

sgg_biz_df_selected = sgg_biz_df[condition]
sgg_biz_df_selected.head(10)

In [ ]:
# 필요없는 칼럼 제거 (열)

columns = ['JACHIGU',"GYE","SAEOPCHESU_1"]

sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final.head()

In [ ]:
# 데이터 프레임의 인덱스 초기화

sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)
sgg_pop_df_final.head()

In [ ]:
import pandas as pd
sgg_biz_df_final.to_excel('./files/sgg_biz.xlsx' ,index = False,)



# 03. 데이터 전처리

## * 가설 1,2을 위한 데이터 전처리 *


* 시군구명, 시군구코드에 서울시 스타벅스 매장 목록, 인구, 사업체 데이터를 추가

In [ ]:
# 서울시 스타벅스 매장 목록이 담긴 파일 불러오기
import pandas as pd
seoul_starbucks = pd.read_excel('./files/seoul_starbucks.xlsx',header=0)
seoul_starbucks.head()

In [ ]:
# 스타벅스 주소 정보에서 시군구명 추출
sgg_names = []
for address in seoul_starbucks['주소']:
    sgg = address.split()[1]
    sgg_names.append(sgg)
seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks.head()

seoul_starbucks.to_excel('./files/seoul_starbucks_list.xlsx', index=False)





* 시군구 목록 데이터 + 시군구별 스타벅스 매장 수, 주민인구 수 , 사업체 수

In [ ]:
import pandas as pd

# 시군구 목록 데이터 불러오기
seoul_sgg = pd.read_excel('./files/seoul_sgg_list.xlsx')
seoul_sgg.head()

In [ ]:
# 스타벅스 매장 목록 데이터 불러오기

seoul_star = pd.read_excel('./files/seoul_starbucks_list.xlsx')
seoul_star.head()

In [ ]:
# 시군구별 스타벅스 매장 수 세기 # pivottable 함수쓰기

pivot_table = seoul_star['시군구명'].value_counts()

pivot_table


In [ ]:
# 서울시 시군구 목록 데이터에 스타벅스 매장 수 데이터를 병합

merge_df = pd.merge(seoul_sgg,pivot_table, on ='시군구명', how='left') # 시군구명을 기준으로 오른쪽파일을 왼쪽 엑셀파일에 합치기

merge_df.rename(columns={'count': '스타벅스_매장 수'}, inplace=True)

merge_df

In [ ]:
# 서울시 시군구별 인구통계 데이터 불러오기

seoul_sgg_pop = pd.read_excel('./files/sgg_pop.xlsx')
seoul_sgg_pop

In [ ]:
# merge_df에 서울 인구수 합치기

merge_df = pd.merge(merge_df,seoul_sgg_pop, on ='시군구명', how='left') # merge_df에서 시군구명 기준으로 주민등록 인구 합치기



merge_df

In [ ]:
import pandas as pd

# 엑셀 파일 읽기
df = pd.read_excel('./files/sgg_biz.xlsx')

# 시군구명을 기준으로 그룹화하여 종사자수와 사업체수를 더하기
grouped_data = df.groupby('시군구명').agg({'종사자수': 'sum', '사업체수': 'sum'}).reset_index()

# 결과 출력
print(grouped_data)


In [ ]:
# 서울시 시군구 목록 데이터에 서울시 시군구별 사업체 수 통계 데이터를 병합



#sgg_biz.xlsx : 시군구별 사업체 현황(가공)

merge_df = pd.merge(merge_df,grouped_data, on ='시군구명', how='left') # merge_df에서 시군구명 기준으로  종사자수, 사업체수 합치기
merge_df

## * 가설 3,4,5을 위한 데이터 전처리 *

* 서울시 상권 분석 서비스의 2023 1분기와 2분기 데이터 추출 및 필요한 컬럼 추출

* 2023년 1분기, 2분기의 데이터 추출

In [ ]:
import pandas as pd
# 데이터 파일 불러오기 
df=pd.read_csv("./files/서울시 상권분석서비스(점포-자치구).csv",encoding="cp949")

#1분기 데이터 전처리 
df_recent1= df[df["기준_년분기_코드"]==20231]
df_Q1= df_recent1[["자치구_코드_명","서비스_업종_코드","유사_업종_점포_수","개업_점포_수","폐업_점포_수"]]
df_Q1.reset_index(drop=True,inplace=True)

df_Q1.head()

In [ ]:
#2분기 데이터 전처리 
df_recent2= df[df["기준_년분기_코드"]==20232]
df_Q2= df_recent2[["자치구_코드_명","서비스_업종_코드","유사_업종_점포_수","개업_점포_수","폐업_점포_수"]]
df_Q2.reset_index(drop=True,inplace=True)

df_Q2.head()

In [ ]:
df_Q2["서비스_업종_외서소"] = df_Q2["서비스_업종_코드"].str.split("0", expand=True)[0]
df_Q2.drop(["서비스_업종_코드"],axis=1,inplace=True)
df_Q2

In [ ]:
df_Q2_bybusiness=df_Q2.groupby(["자치구_코드_명","서비스_업종_외서소"]).sum()
df_Q2_bybusiness.reset_index(inplace=True)
df_Q2_bybusiness.rename(columns={"자치구_코드_명":"시군구명"},inplace=True)
df_Q2_bybusiness

* 2023년 2분기 전체 점포수와 스타벅스 매장을 위한 데이터 전처리

In [ ]:
# "서비스_업종_코드" 열을 가공 CS1:외식업 CS2:서비스업 CS3:소매업
df_Q1["서비스_업종_외서소"] = df_Q1["서비스_업종_코드"].str.split("0", expand=True)[0]
df_Q1.drop(["서비스_업종_코드"],axis=1,inplace=True)
df_Q1

In [ ]:
# "자치구_코드_명"과 "서비스_업종_외서소" 열을 기준으로 그룹화 및 합계 계산
# 외서소: 외식업,서비스업,소매업
df_Q1_bybusiness=df_Q1.groupby(["자치구_코드_명","서비스_업종_외서소"]).sum()
df_Q1_bybusiness.reset_index(inplace=True)
df_Q1_bybusiness

In [ ]:
# "자치구_코드_명" 기준으로 그룹화 후 구별 총점포수 계산 

df_Q1_byareagroup=df_Q1.groupby(["자치구_코드_명"]).sum()
df_Q1_byarea = df_Q1_byareagroup.drop(["서비스_업종_외서소"],axis=1)
df_Q1_byarea

In [ ]:
# 2분기 업종벌 점포수 데이터 

df_Q2_bybusiness.reset_index(inplace=True)
df_Q2_bybusiness.rename(columns={"자치구_코드_명":"시군구명"},inplace=True)
df_Q2_bybusiness

In [ ]:
# 2분기 구별 점포수 데이터 

df_Q2_byareagroup=df_Q2.groupby(["자치구_코드_명"]).sum()
df_Q2_byarea = df_Q2_byareagroup.drop(["서비스_업종_외서소"],axis=1)
df_Q2_byarea

In [ ]:
# 2분기 구별 데이터 프라임에 스타벅스 매장수 및 경위도 열추가 

df_Q2_byarea.reset_index(inplace=True)

df_Q2_byarea.rename(columns={"자치구_코드_명":"시군구명"},inplace=True)
satrbucks_sgg=pd.read_excel("./files/seoul_sgg_stat.xlsx")
df_Q2_byarea_withstar = pd.merge(df_Q2_byarea,satrbucks_sgg,how="left",on="시군구명")
df_Q2_byarea_withstar.drop(["시군구코드","주민득록인구","종사자수","사업체수"],axis=1,inplace=True)
df_Q2_byarea_withstar

In [ ]:
df_Q2_byarea_withstar.to_excel("./files/df_Q2_byarea_withstar가설1.xlsx",index=False)

In [ ]:
df_Q2_byarea_withstar.head()

* 상권 유형별 스타벅스 매장 상관관계를 위한 데이터 전처리

In [ ]:
# 업종별 점포수 데이터 

df_bybusiness_final=pd.read_excel("./files//df_bybusiness_final가설2.xlsx")
df_bybusiness_final.head()

In [ ]:
# 2분기 외식업 점포수 데이터
df_Q2_CS1= df_bybusiness_final[df_bybusiness_final['서비스_업종_외서소'] == 'CS1']
df_Q2_CS1

In [ ]:
# 1분기 외식업 점포수 데이터

df_Q1_CS1 = df_Q1_bybusiness[df_Q1_bybusiness['서비스_업종_외서소'] == 'CS1']
df_Q1_CS1

In [ ]:
# 1&2분기 "개업_점포_수" 및 "폐업_점포_수"의 차이를 구함 

df_Q2_CS1["개업추세"]=df_Q2_CS1["개업_점포_수"]-df_Q1_CS1["개업_점포_수"]
df_Q2_CS1["폐업추세"]=df_Q2_CS1["폐업_점포_수"]-df_Q1_CS1["폐업_점포_수"]
df_Q2_CS1

In [ ]:
# 양수인 경우에는 True를 반환, 음수인 경우에는 False를 반환

def positive_to_true_negative_to_false(value):
    if value>0:
        return True
    else:
        return False
    
df_Q2_CS1["개업수_bool"]=df_Q2_CS1["개업추세"].apply(positive_to_true_negative_to_false)
df_Q2_CS1["폐업수_bool"]=df_Q2_CS1["폐업추세"].apply(positive_to_true_negative_to_false)
df_Q2_CS1

In [ ]:
# "폐업수_bool","개업수_bool" 열의 값을 정수로 변환

df_Q2_CS1["result"] = (df_Q2_CS1["개업수_bool"].astype(int) * 2) + (df_Q2_CS1["폐업수_bool"].astype(int) + 1)
df_Q2_CS1=df_Q2_CS1.sort_values(by="result",ascending=False)
df_Q2_CS1

# 데이터 시각화

### 가설1 데이터 시각화 (주민등록인구 수 별 스타벅스 매장 관계성)

In [ ]:
import folium
import pandas as pd
import json

# 서울시 시군구 행정 경계 지도

sgg_geojson_file_path = './map/seoul_sgg.geojson'

seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')
seoul_sgg_stat['만명당_매장수'] = ((seoul_sgg_stat['스타벅스_매장 수']*10000) / seoul_sgg_stat['주민등록인구']).round(4)

# 서울시 시군구별 주민등록인구수 단계구분도 지도 시각화
seoul_sgg_geo = json.load(open(sgg_geojson_file_path,encoding='utf-8'))
print(type(seoul_sgg_geo))
seoul_sgg_geo

starbucks_boundary = folium.Map(
    location=[37.573050,126.979189],tiles = 'CartoDB dark_matter', zoom_start=11
)

def style_function(feature):                    # 경계선 스타일 지정
    return {
        'opacity':0.7,
        'weight':1,
        'color':'white',
        'fillOpacity':0,
        'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(                                 # 경계선 표시
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(starbucks_boundary)

folium.Choropleth(                                     # 경계지도 만드는 방법
    geo_data=seoul_sgg_geo,
    name='choropleth',  # 지도의 이름
    data=seoul_sgg_stat,                        # 불러올 데이터
    columns=['시군구명', '만명당_매장수'], # 지도의 사용할 열
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='만명당_매장수',
).add_to(starbucks_boundary)


for idx, row in seoul_sgg_stat.iterrows():                  # 차트안에 글자 표시 하는 방법
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {row['만명당_매장수']}",
        icon=folium.DivIcon(icon_size=(150,100), icon_anchor=(7,20),
                            html=f'<div style="font-size: 10pt; color : black">{row["만명당_매장수"]}</div>')
    ).add_to(starbucks_boundary)

starbucks_boundary

# 결과 : 주민등록인구수는 송파구, 강서구, 관악구 순으로 많이 분포함
# 스타벅스 매장 강남구, 서초구, 송파구, 중구, 종로구,영등포구,마포구, 서대문구에 스타벅스 매장이 많이 입점

# 스타벅스 매장수의 단계구분도와 시군구별 주민등록인구수의 단계구분도는 확연히 다름.

In [ ]:
import folium
import pandas as pd
import json

seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')
seoul_sgg_stat.rename(columns={'주민득록인구': '주민등록인구'}, inplace=True)

# 서울시 시군구 행정 경계 지도

sgg_geojson_file_path = './map/seoul_sgg.geojson'


# 서울시 시군구별 주민등록인구수 단계구분도 지도 시각화

seoul_sgg_geo = json.load(open(sgg_geojson_file_path,encoding='utf-8'))
print(type(seoul_sgg_geo))
seoul_sgg_geo

starbucks_bubble = folium.Map(
    location=[37.573050,126.979189],tiles = 'CartoDB dark_matter', zoom_start=11
)




def style_function(feature):                    # 경계선 스타일 지정
    return {
        'opacity':0.7,
        'weight':1,
        'color':'white',
        'fillOpacity':0,
        'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(                                 # 경계선 표시
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(starbucks_bubble)


folium.Choropleth(                                     # 경계지도 만드는 방법
    geo_data=seoul_sgg_geo,
    name='choropleth',  # 지도의 이름
    data=seoul_sgg_stat,
    columns=['시군구명', '주민등록인구'], # 지도의 사용할 열
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='주민등록인구',
).add_to(starbucks_bubble)


for idx, row in seoul_sgg_stat.iterrows():                  # 글자 표시 하는 방법
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {row['주민등록인구']}",
        icon=folium.DivIcon(icon_size=(150,100), icon_anchor=(7,20),
                            html=f'<div style="font-size: 10pt; color : black">{row["주민등록인구"]}</div>')
    ).add_to(starbucks_bubble)



    
starbucks_bubble

# 결과 : 주민등록인구수는 송파구, 강서구, 관악구 순으로 많이 분포함

In [ ]:
import folium
import pandas as pd
import json


sgg_geojson_file_path = './map/seoul_sgg.geojson'


# 서울시 시군구별 주민등록인구수 단계구분도 지도 시각화

seoul_sgg_geo = json.load(open(sgg_geojson_file_path, encoding='utf-8'))



seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')
seoul_sgg_stat.rename(columns={'주민득록인구': '주민등록인구'}, inplace=True)
seoul_sgg_stat['만명당_매장수'] = ((seoul_sgg_stat['스타벅스 매장수']*10000) / seoul_sgg_stat['주민등록인구']).round(4)


# seoul_sgg_stat에서 상위 10퍼센트 데이터프레임 생성
seoul_sgg_stat_top10 = seoul_sgg_stat[seoul_sgg_stat['만명당_매장수'] >= seoul_sgg_stat['만명당_매장수'].quantile(0.9)]


starbucks_bubble = folium.Map(location=[37.5665, 126.9780], zoom_start=11,tiles = 'CartoDB dark_matter')




# 버블지도 그리기
for idx, row in seoul_sgg_stat.iterrows():
    if row['만명당_매장수'] in seoul_sgg_stat_top10['만명당_매장수'].values: # seoul_sgg_stat에 seoul_sgg_stat_top10밸류값이 있으면
        color = 'red'
    else:
        color = 'blue'


    매장수_rounded = f"{row['만명당_매장수']:.4f}" # round 해줌


    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius= 10,
        color='white',
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['시군구명']} {매장수_rounded}",
    ).add_to(starbucks_bubble)


# 지도에 숫자표시
for idx, row in seoul_sgg_stat.iterrows():
    매장수_rounded = f"{row['만명당_매장수']:.4f}"
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {매장수_rounded}",
        icon=folium.DivIcon(icon_size=(150, 100), icon_anchor=(7, 20),
                            html=f'<div style="font-size: 10pt; color: white">{매장수_rounded}</div>')
    ).add_to(starbucks_bubble)


# 경계선
def style_function(feature):
    return {
        'opacity':0.7,
        'weight':1,
        'color':'white',
        'fillOpacity':0,
        'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(starbucks_bubble)

starbucks_bubble


# 결론 : 시군구별 인구 단계구분도에서 중구와 종로구가 서울에서 주민등록인구가 적은 지역으로 표시
# 그러나 인구 1만 명당 스타벅스 매장 수는 가장 많은 것으로 표시
# 가설 1은 틀린 것으로 판단

### 가설2 데이터 시각화 (종사자 수별 스타벅스 매장 관계성)

In [ ]:
import pandas as pd
import json
import folium

seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')


sgg_geojson_file_path =  './map/seoul_sgg.geojson'




seoul_sgg_geo = json.load(open(sgg_geojson_file_path, encoding='utf-8'))



starbucks_boundary = folium.Map(
    location=[37.573050,126.979189],tiles = 'CartoDB dark_matter', zoom_start=11
)




def style_function(feature):                    # 경계선 스타일 지정
    return {
        'opacity':0.7,
        'weight':1,
        'color':'white',
        'fillOpacity':0,
        'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(                                 # 경계선 표시
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(starbucks_boundary)


folium.Choropleth(                                     # 경계지도 만드는 방법
    geo_data=seoul_sgg_geo,
    name='choropleth',  # 지도의 이름
    data=seoul_sgg_stat,
    columns=['시군구명', '종사자수'], # 지도의 사용할 열
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='주민등록인구',
).add_to(starbucks_boundary)


for idx, row in seoul_sgg_stat.iterrows():                  # 글자 표시 하는 방법
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {row['종사자수']}",
        icon=folium.DivIcon(icon_size=(150,100), icon_anchor=(7,20),
                            html=f'<div style="font-size: 10pt; color : black">{row["종사자수"]}</div>')
    ).add_to(starbucks_boundary)



    
starbucks_boundary

In [ ]:
import folium
import pandas as pd
import json

# 서울시 시군구 행정 경계 지도

sgg_geojson_file_path = './map/seoul_sgg.geojson'

seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')
seoul_sgg_stat['종사자만명당_매장수'] = (seoul_sgg_stat['스타벅스 매장수']/ (seoul_sgg_stat['종사자수']/10000)).round(4)


# 서울시 시군구별 종사자수 단계구분도 지도 시각화

seoul_sgg_geo = json.load(open(sgg_geojson_file_path,encoding='utf-8'))
print(type(seoul_sgg_geo))
seoul_sgg_geo

starbucks_boundary = folium.Map(
    location=[37.573050,126.979189],tiles = 'CartoDB dark_matter', zoom_start=11
)




def style_function(feature):                    # 경계선 스타일 지정
    return {
        'opacity':0.7,
        'weight':1,
        'color':'white',
        'fillOpacity':0,
        'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(                                 # 경계선 표시
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(starbucks_boundary)


folium.Choropleth(                                     # 경계지도 만드는 방법
    geo_data=seoul_sgg_geo,
    name='choropleth',  # 지도의 이름
    data=seoul_sgg_stat,                        # 불러올 데이터
    columns=['시군구명', '종사자만명당_매장수'], # 지도의 사용할 열
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='종사자만명당_매장수',
).add_to(starbucks_boundary)


for idx, row in seoul_sgg_stat.iterrows():                  # 차트안에 글자 표시 하는 방법
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {row['종사자만명당_매장수']}",
        icon=folium.DivIcon(icon_size=(150,100), icon_anchor=(7,20),
                            html=f'<div style="font-size: 10pt; color : black">{row["종사자만명당_매장수"]}</div>')
    ).add_to(starbucks_boundary)



    
starbucks_boundary

# 결과 : 주민등록인구수는 송파구, 강서구, 관악구 순으로 많이 분포함
# 스타벅스 매장 강남구, 서초구, 송파구, 중구, 종로구,영등포구,마포구, 서대문구에 스타벅스 매장이 많이 입점

# 스타벅스 매장수의 단계구분도와 시군구별 주민등록인구수의 단계구분도는 확연히 다름.

In [ ]:
import folium
import pandas as pd
import json


seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')


# 종사자 수와 매장수의 비율
seoul_sgg_stat['종사자만명당_매장수'] = (seoul_sgg_stat['스타벅스 매장수']/ (seoul_sgg_stat['종사자수']/10000)).round(4)



sgg_geojson_file_path = './map/seoul_sgg.geojson'
seoul_sgg_geo = json.load(open(sgg_geojson_file_path, encoding='utf-8'))

viz_map_1 = folium.Map(
    location=[37.573050,126.979189],tiles = 'CartoDB dark_matter', zoom_start=11
)

def style_function(feature):                    # 경계선 스타일 지정
    return {
        'opacity':0.7,
        'weight':1,
        'color':'white',
        'fillOpacity':0,
        'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(                                 # 경계선 표시
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(viz_map_1)
   
# 종사자수 만명당 매장 수 기준 상위 10% 추출값

seoul_sgg_stat_top10 = seoul_sgg_stat[seoul_sgg_stat['종사자만명당_매장수'] >= seoul_sgg_stat['종사자만명당_매장수'].quantile(0.9)]





# 버블지도 그리기
for idx, row in seoul_sgg_stat.iterrows():
    if row['종사자만명당_매장수'] in seoul_sgg_stat_top10['종사자만명당_매장수'].values: # seoul_sgg_stat에 seoul_sgg_stat_top10밸류값이 있으면
        color = 'red'
    else:
        color = 'blue'


    매장수_rounded = f"{row['종사자만명당_매장수']:.4f}" # round 해줌


    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius= 10,
        color='white',
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['시군구명']} {매장수_rounded}",
    ).add_to(viz_map_1)


# 지도에 숫자표시
for idx, row in seoul_sgg_stat.iterrows():
    매장수_rounded = f"{row['종사자만명당_매장수']:.4f}"
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {매장수_rounded}",
        icon=folium.DivIcon(icon_size=(150, 100), icon_anchor=(7, 20),
                            html=f'<div style="font-size: 10pt; color: white">{매장수_rounded}</div>')
    ).add_to(viz_map_1)
    
viz_map_1



# 결론: 상위 10프로는 서대문구 종로구 마포구

# 사업체 종사자 수와 스타벅스 매장 수는 대체로 비례함

# 서대문구의 경우 사업체 종사자 수가 많지 않은 지역임 직장인이 많은 곳에 스타벅스 매장이 많을 것이라는 가설 2가 의미가 있음
                    
         

In [ ]:
# 종사자수 만명당 매장 수 기준 상위 10% 추출값

seoul_sgg_stat_top10 = seoul_sgg_stat[seoul_sgg_stat['종사자만명당_매장수'] >= seoul_sgg_stat['종사자만명당_매장수'].quantile(0.9)]





# 버블지도 그리기
for idx, row in seoul_sgg_stat.iterrows():
    if row['종사자만명당_매장수'] in seoul_sgg_stat_top10['종사자만명당_매장수'].values: # seoul_sgg_stat에 seoul_sgg_stat_top10밸류값이 있으면
        color = 'red'
    else:
        color = 'blue'


    매장수_rounded = f"{row['종사자만명당_매장수']:.4f}" # round 해줌


    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius= 10,
        color='white',
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['시군구명']} {매장수_rounded}",
    ).add_to(viz_map_1)


# 지도에 숫자표시
for idx, row in seoul_sgg_stat.iterrows():
    매장수_rounded = f"{row['종사자만명당_매장수']:.4f}"
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {매장수_rounded}",
        icon=folium.DivIcon(icon_size=(150, 100), icon_anchor=(7, 20),
                            html=f'<div style="font-size: 10pt; color: white">{매장수_rounded}</div>')
    ).add_to(viz_map_1)
    
viz_map_1



# 결론: 상위 10프로는 서대문구 종로구 마포구

# 사업체 종사자 수와 스타벅스 매장 수는 대체로 비례함

# 서대문구의 경우 사업체 종사자 수가 많지 않은 지역임 직장인이 많은 곳에 스타벅스 매장이 많을 것이라는 가설 2가 의미가 있음

### 가설3 데이터 시각화 (전체 점포 수와 스타벅스 매장 관계성)

In [ ]:
# 가설 1: 시군구별 총점포수 및 스타박스 매장수 비교 
# 총점포 수 
import pandas as pd
import matplotlib
matplotlib.rcParams['font.family'] = 'AppleGothic' 
import matplotlib.pyplot as plt
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['axes.unicode_minus'] = False

df = df_Q2_byarea_withstar

fig, ax1 = plt.subplots(figsize=(10, 5))


# 총점포 수: 막대 그래프로 표현/ 스타박스 매장수: 선 그래프 그리기 
bars = ax1.bar(df['시군구명'], df['유사_업종_점포_수'], color='#ffadad')
ax1.set_xlabel('지역')
ax1.set_ylabel('지역별 전체 점포 수')
ax1.bar(df['시군구명'], df['유사_업종_점포_수'],color='#ffadad')
ax1.tick_params(axis='y')
plt.xticks(rotation=90) 

ax2 = ax1.twinx() # x축과 공유하는 새로운 y축을 만듦
ax2.set_ylabel('스타벅스 매장수') #label 표시
ax2.plot(df['시군구명'], df['스타벅스 매장수'], marker='o',color='#a0c4ff') # x 시군구명, y 스타벅스 매장수
ax2.tick_params(axis='y') # y축에 대한 틱 -> 틱 라벨의 색상, 라벨의 크기 조절 가능

plt.tight_layout() # 플롯의 여백 최적화

plt.show()

# 상권 내에 점포수와 스타벅스 매장의 관계가 어느정도 유의미한 관계를 가짐 

In [ ]:
# 스타벅스 매장과 지역별 전체 점포 수를 산점도 그래프로 

plt.figure(figsize=(10, 5))
plt.scatter(df['유사_업종_점포_수'], df['스타벅스 매장수'], color='blue', alpha=0.5)
plt.title('시군구별 점포 수와 스타벅스 매장 수의 관계')
plt.xlabel('전체 점포 수')
plt.ylabel('스타벅스 매장수')

# 각 점에 시군구명 표시
for i, txt in enumerate(df['시군구명']):
    plt.annotate(txt, (df['유사_업종_점포_수'][i], df['스타벅스 매장수'][i]), fontsize=8, ha='right', va='bottom')

plt.grid(True)
plt.show()

In [ ]:
# 시군구별 전체 점포수를 경계선 지도로 나타내는 코드


import pandas as pd
import folium
import json


merge_outer1 = df_Q2_byarea_withstar


ssg_geojson_file_path = "./map/seoul_sgg.geojson"
seoul_sgg_geo = json.load(open(ssg_geojson_file_path,encoding='utf-8'))
print(type(seoul_sgg_geo))
seoul_sgg_geo

starbucks_store_map = folium.Map(
    location=[37.573050,126.979189], zoom_start=11
)

def style_function(feature):
    return {
        'opacity':0.7,
        'weight':1,
        'color':'black',
        'fillOpacity':0,
        # 'dashArray':'5, 5' # 실선과 점선의 길이
    }
    
folium.GeoJson(
    seoul_sgg_geo,
    style_function=style_function
    
).add_to(starbucks_store_map)

folium.Choropleth(                                     # 경계지도 만드는 방법
    geo_data=seoul_sgg_geo,
    name='choropleth',  # 지도의 이름
    data=merge_outer1,
    columns=['시군구명', '유사_업종_점포_수'], # 지도의 사용할 열
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='유사_업종_점포_수',
).add_to(starbucks_store_map)

for idx, row in merge_outer1.iterrows():                  # 글자 표시 하는 방법
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['시군구명']}: {row['유사_업종_점포_수']}",
        icon=folium.DivIcon(icon_size=(150,100), icon_anchor=(7,20),
                            html=f'<div style="font-size: 10pt; color : black">{row["유사_업종_점포_수"]}</div>')
    ).add_to(starbucks_store_map)


for idx, row in df.iterrows():              # 버블지도 만들기 
    
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=row['스타벅스 매장수']*0.3,  
        color='#00704A',
        fill=True,
        fill_color='#00704A',
        fill_opacity=1,
        popup=f"{row['시군구명']}<br>스타벅스 매장수: {row['스타벅스 매장수']}",
    ).add_to(starbucks_store_map)
    

starbucks_store_map

In [ ]:
# 지역별 전체 점포 수와 스타벅스 매장의 상관관계 분석 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['font.family'] = 'AppleGothic' 

matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['axes.unicode_minus'] = False

corr = merge_outer1['유사_업종_점포_수'].corr(merge_outer1['스타벅스 매장수'])

print(corr)

plt.figure(figsize=(10,5))
plt.title('전체 점포수와 스타벅스 매장 수의 관계')
sns.scatterplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=merge_outer1, label=f'점포수 (상관 계수: {corr:.2f})')
plt.xlabel('전체 점포수')
plt.ylabel('스타벅스 매장수')

sns.regplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=merge_outer1,color='blue')
plt.xlabel('전체 점포수')
plt.ylabel('스타벅스 매장수')
plt.show()

### 가설4 데이터 시각화 (업종별 점포 수와 스타벅스 매장 관계성)

In [ ]:
# 가설 2: 업종별 점포수 및 스타박스 매장수 비교 

import pandas as pd


df = pd.read_excel('./files/df_bybusiness_final가설2.xlsx')
df.reset_index
df.head()    

In [ ]:
# 서비스 업종 코드를 숫자별로 나눔

df1 = df[df['서비스_업종_외서소'].str[2] == '1']  # 1이면 외식업

df2 = df[df['서비스_업종_외서소'].str[2] == '2']  # 2이면 서비스업

df3 = df[df['서비스_업종_외서소'].str[2] == '3']  # 3이면 외식업



In [ ]:
# 업종별 점포수와 스타벅스 매장 수의 관계를 여러 그래프로 만들기

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['axes.unicode_minus'] = False



fig, axs = plt.subplots(3, 1, figsize=(15, 15))

# 첫 번째 서브플롯
axs[0].set_title("외식업과 스타벅스 매장의 관계")
axs[0].set_ylabel('외식업 점포수')
axs[0].bar(df1['시군구명'], df1['유사_업종_점포_수'], color='#ffadad')
axs[0].tick_params(axis='y')
ax2 = axs[0].twinx()  # x축과 공유하는 새로운 y축을 만듦
ax2.set_ylabel('스타벅스 매장 수')  # label 표시
ax2.plot(df1['시군구명'], df1['스타벅스 매장수'], marker='o', color='#a0c4ff')  # x 시군구명, y 스타벅스 매장수
ax2.tick_params(axis='y')  # y축에 대한 틱 -> 틱 라벨의 색상, 라벨의 크기 조절 가능
axs[0].tick_params(axis='x', rotation=90)



# 두 번째 서브플롯
axs[1].set_title("서비스업과 스타벅스 매장의 관계")
axs[1].set_ylabel('서비스업 점포수')
axs[1].bar(df2['시군구명'], df2['유사_업종_점포_수'], color='#A3B5FF')
axs[1].tick_params(axis='y')
plt.xticks(rotation=90) 
ax3 = axs[1].twinx()  # x축과 공유하는 새로운 y축을 만듦
ax3.set_ylabel('스타벅스 매장 수')  # label 표시
ax3.plot(df2['시군구명'], df2['스타벅스 매장수'], marker='o', color='#757575')  # x 시군구명, y 스타벅스 매장수
ax3.tick_params(axis='y')  # y축에 대한 틱 -> 틱 라벨의 색상, 라벨의 크기 조절 가능
axs[1].tick_params(axis='x', rotation=90)


# 서번째 서브플롯
axs[2].set_title("소매업과 스타벅스 매장의 관계")
axs[2].set_xlabel('지역')
axs[2].set_ylabel('소매업 점포수')
axs[2].bar(df3['시군구명'], df3['유사_업종_점포_수'], color='#7BFFB9')
axs[2].tick_params(axis='y')
ax4 = axs[2].twinx()  # x축과 공유하는 새로운 y축을 만듦
ax4.set_ylabel('스타벅스 매장 수')  # label 표시
ax4.plot(df3['시군구명'], df3['스타벅스 매장수'], marker='o', color='#757575')  # x 시군구명, y 스타벅스 매장수
ax4.tick_params(axis='y')  # y축에 대한 틱 -> 틱 라벨의 색상, 라벨의 크기 조절 가능
axs[2].tick_params(axis='x', rotation=90)
plt.tight_layout()  # 플롯의 여백 최적화

plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 상관 계수 계산
correlation = df1['유사_업종_점포_수'].corr(df1['스타벅스 매장수'])
correlation2 = df2['유사_업종_점포_수'].corr(df2['스타벅스 매장수'])
correlation3 = df3['유사_업종_점포_수'].corr(df3['스타벅스 매장수'])

# 산점도 그리기
plt.figure(figsize=(8, 6))
sns.scatterplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=df1, label=f'외식업 (상관 계수: {correlation:.3f})', color='red')
sns.scatterplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=df2, label=f'서비스업 (상관 계수: {correlation2:.3f})', color='blue')
sns.scatterplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=df3, label=f'소매업 (상관 계수: {correlation3:.3f})', color='green')

plt.title('스타벅스 매장의 상관 계수 산점도')
plt.xlabel('업종별 점포수')
plt.ylabel('스타벅스 매장 수')
plt.legend()

plt.show()

In [ ]:
# 서브플롯을 이용하여 업종별 점포수와 스타벅스 매장 수의 관계의 그래프들을 한 곳에 만들기

import seaborn as sns
import matplotlib.pyplot as plt


fig, axs = plt.subplots(3, 1, figsize=(10, 10))


sns.regplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=df1, ax=axs[0])
axs[0].set_xlabel('외식업 매장의 수')
axs[0].set_title('외식업 점포수와 스타벅스 매장 수의 관계')


sns.regplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=df2, ax=axs[1])
axs[1].set_xlabel('서비스업 매장의 수')
axs[1].set_ylabel('서비스업 매장의 수')
axs[1].set_title('서비스업 점포수와 스타벅스 매장 수의 관계')

sns.regplot(x='유사_업종_점포_수', y='스타벅스 매장수', data=df3, ax=axs[2])
axs[2].set_xlabel('소매업 매장의 수')
axs[2].set_title('소매업 점포수와 스타벅스 매장 수의 관계')


plt.tight_layout()


plt.show()


### 가설5 데이터 시각화 (2032년 분기별 상권유형과 스타벅스 매장 수의 관계성 )

In [ ]:
import folium
import json

# 서울 지도 및 각구의 경계선 그리기 
geo_path = './map/seoul_sgg.geojson'
geo_data = json.load(open(geo_path, encoding='utf-8'))


seoul_map = folium.Map(location=[37.55,126.98], zoom_start=9)
def style_function(feature):
    return {
        "opacity": 0.7,
        "weight": 1,
        "color": "white",
        "fillOpacity": 0,
        "dashArray": "5,5"}
    
folium.GeoJson(
    geo_data,
    style_function=style_function,
).add_to(seoul_map)


# 임계값 설정 
threshold_scale = [0.5,1.5,3.1,3.9] 

folium.Choropleth(geo_data=geo_data,    # 지도 경계
                  data = df_Q2_CS1,      # 표시하려는 데이터
                  columns = ["시군구명","result"],  # 열 지정
                  fill_color='YlOrRd', 
                  fill_opacity=0.7, 
                  line_opacity=0.3,
                  key_on='feature.properties.SIG_KOR_NM').add_to(seoul_map)

# 지도에 상권 유행 표시 
#result_labels = {1:"FF_안정한 상권_마케팅 필요",3:"TF_확장중인 상권" , 4:"TT_상권변화가 활발_입지선정(불안전한 상권)"}
result_labels = {1:"안정한\n상권",2:"쇠퇴한\n상권",3:"확장중인\n상권", 4:"상권변화가활발\n(불안정)"}

for idx, row in df_Q2_CS1.iterrows():
    
    result_value = row["result"]
    result_label = result_labels.get(result_value,"Unknown")
    
    
    folium.Marker(
        location=[row["위도"],row["경도"]],
        popup = f"{row['시군구명']}:{result_label}",
        icon = folium.DivIcon(icon_size=(150,100),icon_anchor=(7,20),
                             html=f'<div style="font-size: 8pt; color : black; fill_opacity: 0.7">{result_label}</div>')
    ).add_to(seoul_map)

# 지도에 스타벅스 매장 수에 따라 원을 표시
for name,lat,lng,stores in zip(df_Q2_CS1["시군구명"], df_Q2_CS1["위도"], df_Q2_CS1["경도"],df_Q2_CS1["스타벅스 매장수"]):

    folium.CircleMarker([lat, lng],
                        radius=stores * 0.2,  # 원의 크기
                        color='#00704A',         # 원의 둘레 색상
                        fill=True,
                        fill_color='#00704A',    # 원을 채우는 색
                        fill_opacity=1, # 투명도    
                        popup=name
    ).add_to(seoul_map)
seoul_map

In [ ]:
df_Q2_CS1.to_excel("./files/가설3.xlsx",index=False)

### 가설6 데이터 시각화 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc("font",family = "Apple SD Gothic Neo")

# 상관 계수 계산
# 4번 상권:변화가 매우활발한(불안정한) 상권
result_4_data = df_Q2_CS1[df_Q2_CS1['result'] == 4]
correlation4 = result_4_data['유사_업종_점포_수'].corr(result_4_data['스타벅스 매장수'])
# 3번 상권: 확장중인 상권
result_3_data = df_Q2_CS1[df_Q2_CS1['result'] == 3]
correlation3 = result_3_data['유사_업종_점포_수'].corr(result_3_data['스타벅스 매장수'])

plt.figure(figsize=(25, 8))


# 산포도 그리기 
plt.subplot(131)
sns.scatterplot(x="유사_업종_점포_수", y="스타벅스 매장수", data=result_4_data, label=f"불안정한 상권 (상관 계수: {correlation4:.3f})", color="red")
sns.scatterplot(x="유사_업종_점포_수", y="스타벅스 매장수", data=result_3_data, label=f"확장중인 상권 (상관 계수: {correlation3:.3f})", color="blue")
plt.title("외식업 매장의 수와 스타벅스 매장 수의 관계")
plt.xlabel("업종별 점포수")
plt.ylabel("스타벅스 매장 수")
plt.legend()

# 불안정한 상권의 산포도 corr =0.79
plt.subplot(132)
sns.regplot(x="유사_업종_점포_수", y="스타벅스 매장수", data=result_4_data)
plt.title("변화가 매우활발한(불안정한) 상권의 외식업 점포수와 스타벅스 매장 수의 관계 corr=0.79")
plt.xlabel("외식업 매장의 수")

# 확장중인 상권의 산포도 corr = 0.94
plt.subplot(133)
sns.regplot(x="유사_업종_점포_수", y="스타벅스 매장수", data=result_3_data)
plt.title("확장중인 상권의 외식업 점포수와 스타벅스 매장 수의 관계 corr=0.94")
plt.xlabel("외식업 매장의 수")

plt.tight_layout()
plt.show()
